In [5]:
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
#llm=Ollama(model="llama2")
from transformers import pipeline
import nltk

# Download NLTK data for sentiment analysis
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer

# Initialize sentiment analyzer
sentiment_analyzer = SentimentIntensityAnalyzer()

# Initialize the model for LLaMA
llama_model = Ollama(model='llama2')

# Summarization and NLP functions
def summarize_text(text):
    """Function to summarize text using LLaMA model"""
    prompt = f"Summarize this conversation: {text}"
    summary = llama_model(prompt)
    return summary

def analyze_sentiment(text):
    """Function to analyze sentiment of the text"""
    sentiment_scores = sentiment_analyzer.polarity_scores(text)
    sentiment = 'Positive' if sentiment_scores['compound'] > 0 else 'Negative'
    return sentiment

async def start(update: Update, context):
    """Start command for the bot"""
    await update.message.reply_text('Hello! I am a chatbot to summarize group chats and analyze sentiment. Send me a message!')

async def summarize(update: Update, context):
    """Summarize user messages"""
    user_message = update.message.text
    summary = summarize_text(user_message)
    await update.message.reply_text(f"Summary: {summary}")

async def sentiment(update: Update, context):
    """Analyze sentiment of user messages"""
    user_message = update.message.text
    sentiment = analyze_sentiment(user_message)
    await update.message.reply_text(f"Sentiment: {sentiment}")

# Main function to run the bot
import asyncio

async def main():
    app = ApplicationBuilder().token('7809549813:AAFNAsixE_dZ-rfcgPdl_EeqGQP49l1sCgg').build()

    # Commands
    app.add_handler(CommandHandler("start", start))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, summarize))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, sentiment))

    async with app:
        await app.updater.start_polling()

# Main execution logic
if __name__ == '__main__':
    try:
        loop = asyncio.get_running_loop()
        # If there's a running loop, use it to run the main coroutine
        if loop.is_running():
            print("Event loop already running. Using existing loop...")
            task = loop.create_task(main())
        else:
            loop.run_until_complete(main())
    except RuntimeError:  # If no event loop is running
        asyncio.run(main())


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\divag\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
Task exception was never retrieved
future: <Task finished name='Task-6' coro=<main() done, defined at C:\Users\divag\AppData\Local\Temp\ipykernel_24764\1740540534.py:51> exception=AttributeError("'Application' object has no attribute 'start_polling'")>
Traceback (most recent call last):
  File "C:\Users\divag\AppData\Local\Temp\ipykernel_24764\1740540534.py", line 59, in main
    await app.start_polling()
          ^^^^^^^^^^^^^^^^^
AttributeError: 'Application' object has no attribute 'start_polling'


Event loop already running. Using existing loop...


Exception happened while polling for updates.
Traceback (most recent call last):
  File "c:\Users\divag\AppData\Local\Programs\Python\Python312\Lib\site-packages\telegram\request\_baserequest.py", line 333, in _request_wrapper
    code, payload = await self.do_request(
                    ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\divag\AppData\Local\Programs\Python\Python312\Lib\site-packages\telegram\request\_httpxrequest.py", line 267, in do_request
    raise RuntimeError("This HTTPXRequest is not initialized!")
RuntimeError: This HTTPXRequest is not initialized!

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\divag\AppData\Local\Programs\Python\Python312\Lib\site-packages\telegram\ext\_updater.py", line 753, in _network_loop_retry
    if not await do_action():
           ^^^^^^^^^^^^^^^^^
  File "c:\Users\divag\AppData\Local\Programs\Python\Python312\Lib\site-packages\telegram\ext\_updater.py", line 747, in do_

In [8]:
import json
import sqlite3
from datetime import datetime
from telegram import Update, Message
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters, CallbackContext
from dateutil import parser as date_parser
import asyncio

# Replace 'YOUR_TOKEN' with your bot's API token
TOKEN = '7809549813:AAFNAsixE_dZ-rfcgPdl_EeqGQP49l1sCgg'

# Initialize SQLite database
def init_db():
    conn = sqlite3.connect('chat_history.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS messages (
            id INTEGER PRIMARY KEY,
            text TEXT,
            date TEXT,
            user TEXT
        )
    ''')
    conn.commit()
    conn.close()

# Function to store incoming messages in the database
def store_message(message: Message):
    conn = sqlite3.connect('chat_history.db')
    cursor = conn.cursor()
    cursor.execute('''
        INSERT INTO messages (text, date, user) VALUES (?, ?, ?)
    ''', (message.text, message.date.isoformat(), message.from_user.username if message.from_user.username else message.from_user.full_name))
    conn.commit()
    conn.close()

# Function to extract chats within date range
def get_chats_in_date_range(start_date, end_date):
    conn = sqlite3.connect('chat_history.db')
    cursor = conn.cursor()
    cursor.execute('''
        SELECT * FROM messages WHERE date BETWEEN ? AND ?
    ''', (start_date.isoformat(), end_date.isoformat()))
    rows = cursor.fetchall()
    conn.close()
    
    # Convert rows to a list of dictionaries
    return [{'id': row[0], 'text': row[1], 'date': row[2], 'user': row[3]} for row in rows]

# /summarize command handler
async def summarize(update: Update, context: CallbackContext) -> None:
    try:
        if len(context.args) != 2:
            await update.message.reply_text('Please provide two dates in the format: /summarize <date1> <date2>')
            return

        # Parse the dates from the command arguments
        date1 = date_parser.parse(context.args[0]).astimezone()
        date2 = date_parser.parse(context.args[1]).astimezone()

        # Filter chats between the specified date range
        filtered_chats = get_chats_in_date_range(date1, date2)

        # Convert filtered chats to JSON and send back
        if filtered_chats:
            json_chats = json.dumps(filtered_chats, default=str, indent=2)
            await update.message.reply_text(f"Chats between {date1} and {date2}:\n{json_chats}")
        else:
            await update.message.reply_text('No chats found in the specified date range.')

    except Exception as e:
        await update.message.reply_text(f"Error: {str(e)}")

# Message handler to capture all messages
async def handle_message(update: Update, context: CallbackContext) -> None:
    store_message(update.message)

# Check if an event loop is already running
if __name__ == '__main__':
    init_db()  # Initialize the database
    
    # Initialize the application
    application = ApplicationBuilder().token(TOKEN).build()

    # Register the /summarize command handler
    application.add_handler(CommandHandler('summarize', summarize))

    # Register the message handler to capture all messages
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    # Start polling for updates
    application.run_polling()


2024-09-30 17:46:40,279 - asyncio - ERROR - Task exception was never retrieved
future: <Task finished name='Task-38' coro=<main() done, defined at C:\Users\divag\AppData\Local\Temp\ipykernel_24764\1222214022.py:10> exception=InvalidToken('The token `YOUR_TELEGRAM_BOT_TOKEN` was rejected by the server.')>
Traceback (most recent call last):
  File "c:\Users\divag\AppData\Local\Programs\Python\Python312\Lib\site-packages\telegram\_bot.py", line 777, in initialize
    await self.get_me()
  File "c:\Users\divag\AppData\Local\Programs\Python\Python312\Lib\site-packages\telegram\ext\_extbot.py", line 1920, in get_me
    return await super().get_me(
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\divag\AppData\Local\Programs\Python\Python312\Lib\site-packages\telegram\_bot.py", line 909, in get_me
    result = await self._post(
             ^^^^^^^^^^^^^^^^^
  File "c:\Users\divag\AppData\Local\Programs\Python\Python312\Lib\site-packages\telegram\_bot.py", line 623, in _post
    return await

Event loop already running. Using existing loop...


2024-09-30 17:46:41,636 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7809549813:AAFNAsixE_dZ-rfcgPdl_EeqGQP49l1sCgg/getMe "HTTP/1.1 200 OK"


In [21]:
import os

token_bot = os.getenv('TELEGRAM_BOT_TOKEN')  # Make sure to set this environment variable


In [3]:
import logging
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, ContextTypes
import asyncio

# Set up logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

# Define the /start command handler
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text('Hello! I am your bot.')

# Main function to run the bot
async def main():
    app = ApplicationBuilder().token('7809549813:AAFNAsixE_dZ-rfcgPdl_EeqGQP49l1sCgg').build()  # Replace with your actual token
    app.add_handler(CommandHandler("start", start))

    try:
        logging.info("Bot started polling.")
        await app.run_polling()  # Start polling and keep the bot running
    except Exception as e:
        logging.error(f"An error occurred: {e}")
    finally:
        await app.shutdown()  # Ensure proper shutdown of the application

# Run the main function
if __name__ == '__main__':
    try:
        loop = asyncio.get_running_loop()
        if loop.is_running():
            print("Event loop already running. Using existing loop...")
            task = loop.create_task(main())  # Create a new task in the existing loop
        else:
            loop.run_until_complete(main())  # Run the main function if no loop is running
    except RuntimeError:  # If no event loop is running
        asyncio.run(main())


Event loop already running. Using existing loop...


2024-09-30 18:09:43,589 - root - INFO - Bot started polling.
2024-09-30 18:09:43,589 - root - ERROR - An error occurred: Cannot close a running event loop
C:\Users\divag\AppData\Local\Temp\ipykernel_14996\2179091444.py:25: RuntimeWarning: coroutine 'Application.shutdown' was never awaited
  logging.error(f"An error occurred: {e}")
C:\Users\divag\AppData\Local\Temp\ipykernel_14996\2179091444.py:25: RuntimeWarning: coroutine 'Application.initialize' was never awaited
  logging.error(f"An error occurred: {e}")


In [6]:
import logging
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, ContextTypes
import asyncio

# Set up logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

# Define the /start command handler
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text('Hello! I am your bot.')

# Main function to run the bot
async def main():
    app = ApplicationBuilder().token('7809549813:AAFNAsixE_dZ-rfcgPdl_EeqGQP49l1sCgg').build()  # Replace with your actual token
    app.add_handler(CommandHandler("start", start))

    try:
        logging.info("Bot started polling.")
        await app.run_polling()  # Start polling and keep the bot running
    except Exception as e:
        logging.error(f"An error occurred: {e}")
    finally:
        await app.shutdown()  # Ensure proper shutdown of the application

# Run the bot function
def run_bot():
    loop = asyncio.get_event_loop()  # Get the running event loop
    if loop.is_running():
        print("Event loop already running. Creating a task for the bot...")
        task = loop.create_task(main())  # Create a task in the existing loop
    else:
        print("Starting the bot...")
        loop.run_until_complete(main())  # Run the main function if no loop is running

if __name__ == '__main__':
    run_bot()


Event loop already running. Creating a task for the bot...


2024-09-30 18:11:55,313 - root - INFO - Bot started polling.
2024-09-30 18:11:55,314 - root - ERROR - An error occurred: Cannot close a running event loop
C:\Users\divag\AppData\Local\Temp\ipykernel_14996\1487333407.py:25: RuntimeWarning: coroutine 'Application.shutdown' was never awaited
  logging.error(f"An error occurred: {e}")
C:\Users\divag\AppData\Local\Temp\ipykernel_14996\1487333407.py:25: RuntimeWarning: coroutine 'Application.initialize' was never awaited
  logging.error(f"An error occurred: {e}")


In [4]:
import requests
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import os

# Download NLTK data for sentiment analysis
nltk.download('vader_lexicon')
sentiment_analyzer = SentimentIntensityAnalyzer()

# Groq API Setup
#GROQ_API_URL = "https://api.groq.com/v1/summarize"
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

def summarize_text_with_groq(text):
    """Function to summarize text using Groq API."""
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "text": text,
        "length": "short"  # You can adjust the length: short, medium, long
    }
    response = requests.post(GROQ_API_URL, headers=headers, json=data)

    if response.status_code == 200:
        summary = response.json().get('summary', '')
        return summary
    else:
        return "Error: Unable to summarize the text."

def analyze_sentiment(text):
    """Function to analyze sentiment of the text."""
    sentiment_scores = sentiment_analyzer.polarity_scores(text)
    sentiment = 'Positive' if sentiment_scores['compound'] > 0 else 'Negative'
    return sentiment

async def start(update: Update, context):
    """Start command for the bot"""
    await update.message.reply_text('Hello! I am ChatSift. Send me a message to summarize and analyze sentiment!')

async def summarize(update: Update, context):
    """Summarize user messages"""
    user_message = update.message.text
    summary = summarize_text_with_groq(user_message)
    await update.message.reply_text(f"Summary: {summary}")

async def sentiment(update: Update, context):
    """Analyze sentiment of user messages"""
    user_message = update.message.text
    sentiment = analyze_sentiment(user_message)
    await update.message.reply_text(f"Sentiment: {sentiment}")

# Main function to run the bot
async def main():
    app = ApplicationBuilder().token('YOUR_TELEGRAM_BOT_TOKEN').build()

    # Commands
    app.add_handler(CommandHandler("start", start))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, summarize))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, sentiment))

    await app.start_polling()

if __name__ == '__main__':
    import asyncio
    asyncio.run(main())


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\divag\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


RuntimeError: asyncio.run() cannot be called from a running event loop